In [26]:
# Libraries
import pandas as pd
input_folder_path = "DataSource/Input/"
output_folder_path = "DataSource/Output/"

### Sales Data Process

In [27]:
file_name = "Sales.xlsx"
df = pd.read_excel(input_folder_path + file_name, skiprows=4)

# Reset the index
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df = df.reset_index(drop=True)
df = df[['Game Title', 'Item', 'Sales (USD)']]
df.dropna(subset = ['Game Title'], inplace = True)
df = df.groupby('Game Title')['Sales (USD)'].sum().reset_index()
display(df)

,Game Title,Sales (USD)
0,- No Game Title -,1313.96
1,03-2017:Genso Wanderer:Genso Wanderer PS4,0
2,03-2017:Genso Wanderer:Genso Wanderer PS4 DIGITAL,50.65
3,03-2017:Genso Wanderer:Genso Wanderer PS4 PAL,58.72
4,03-2017:Operation Abyss-NTL:Operation Abyss STEAM,82.68
...,...,...
392,Yomawari Night Alone,3.52
393,Yomawari Night Alone PSV PAL,35.76
394,Yomawari Night Alone PSV US,34.57
395,Ys Memories of Celceta PSV PAL,50.78


### COGS data process

In [28]:
# Data Cleanup
COGS_df = pd.read_csv(input_folder_path + "COGS.csv", skiprows=4)
COGS_df.columns = COGS_df.iloc[0]
COGS_df = COGS_df.drop(COGS_df.index[0])
COGS_df = COGS_df.reset_index(drop=True)
COGS_df = COGS_df.dropna(subset = ['Game Title: Name'])


# Grouping by title
COGS_df['Game Title: Name'] = COGS_df['Game Title: Name'].str.replace(' : ', ':', regex = False)
COGS_df['Amount'] = COGS_df['Amount'].str.replace('$', '').str.replace(',', '').str.strip().astype(float)
COGS_df = COGS_df.rename(columns = {'Amount': 'COGS Expense'})
COGS_df = COGS_df.rename(columns = {'Game Title: Name': 'Game Title'})
COGS_df = COGS_df.groupby('Game Title')['COGS Expense'].sum().reset_index()

display(COGS_df)

,Game Title,COGS Expense
0,- No Game Title -,17006.00
1,2024-09_REYNATIS EU Replace,2261.52
2,Cardcaptor Sakura,285.66
3,Disgaea 6 Complete PS4,160.45
4,Disgaea 6 Complete PS5,187.54
5,Disgaea 7 NSW,2670.77
6,Disgaea 7 PS4,184.14
7,Disgaea 7 PS5,551.24
8,Fallen Legion 2 XB1,12.61
9,Goods,8360.38


### Capitalized Expense Process

In [29]:
# clean up
CapExp_df = pd.read_excel(input_folder_path + "Capitalized_Expense.xlsx")
CapExp_df['Capitalized Expense Amount (Debit + Credit)'] = CapExp_df['Amount (Debit)'] + CapExp_df['Amount (Credit)']
CapExp_df.dropna(subset = ['Game Title'], inplace = True)

# grouping by title
CapExp_df = CapExp_df.groupby('Game Title')['Capitalized Expense Amount (Debit + Credit)'].sum().reset_index()
CapExp_df['Game Title'] = CapExp_df['Game Title'].str.replace(' : ', ':', regex = False)
display(CapExp_df)

,Game Title,Capitalized Expense Amount (Debit + Credit)
0,03-2023:Trails through Daybreak (PC):Trails th...,81557.49
1,03-2024:Trails through Daybreak,240404.69
2,03-2025:REYNATIS,35608.15


### AD Promotion Data Process

In [30]:
# clean up
AD_df = pd.read_excel(input_folder_path + "AD_Promotion.xlsx", skiprows=4)
AD_df.columns = AD_df.iloc[0]
AD_df = AD_df.drop(AD_df.index[0])
AD_df = AD_df.reset_index(drop=True)
AD_df = AD_df.dropna(subset = ['Game Title: Name'])

# grouping by title
AD_df = AD_df.groupby('Game Title: Name')['Amount'].sum().reset_index()
AD_df = AD_df.rename(columns = {'Amount': 'AD Promotion'})
AD_df = AD_df.rename(columns = {'Game Title: Name': 'Game Title'})

display(AD_df)

,Game Title,AD Promotion
0,- No Game Title -,4495.77
1,2024-07_Anime Expo 2024,386.02
2,2024-08_PAX West,53860.14
3,Phantom Brave 1 PS5,1058.97
4,Phantom Brave 2,9.25
5,Ys X: Nordics,2960


### Royalty Expense Process

In [31]:
# get data
Royalty_df = pd.read_excel(input_folder_path + "Royalty.xlsx", sheet_name='Import_Expense')
Royalty_df.rename(columns = {'game_title': 'Game Title'}, inplace = True)
Royalty_df['Game Title'] = Royalty_df['Game Title'].str.replace(' : ', ':', regex = False)
Royalty_df.dropna(subset = ['Game Title'], inplace = True)


#grouping by title
Royalty_df = Royalty_df.groupby('Game Title')['sales'].sum().reset_index()
Royalty_df.rename(columns = {'sales': 'Royalty Expense'})

display(Royalty_df)

,Game Title,sales
0,03-2017:Genso Wanderer:Genso Wanderer PS4,41.98
1,03-2017:Genso Wanderer:Genso Wanderer PS4 DIGITAL,1.39
2,03-2017:Genso Wanderer:Genso Wanderer PS4 PAL,1.29
3,03-2017:Genso Wanderer:Genso Wanderer PSV,22.38
4,03-2017:Operation Abyss-NTL:Operation Abyss STEAM,641.60
...,...,...
208,Trails of Cold Steel PSV PAL,124.10
209,Trinity Universe-DLC US,5.56
210,Under Night In Birth PS3 PAL,64.17
211,Way of the Samurai 4 PS3 EU,58.96
